In [1]:
import os
import numpy as np
import pandas as pd
from skimage import io
import util2
import cv2


parent_dir = r"/home/charlie/projects/entropy/input/"


# Create dictionaries to store the results
#active_stats: list[tuple[tuple, float]]
#inactive_stats: list[tuple[tuple, float]]

faz_area = {}
faz_circularity = {}
vessel_density = {}
vessel_length_density = {}


#第一層 列出前處理方式種類
for dirname in os.listdir(parent_dir):
    if (dirname.endswith(".DS_Store")):
        pass
    
    if(dirname == 'binary' or 'binary2'):
        pass
    #each image has up to two parameters, use 'parameter' list to store them
    parameter = []
    
    #each image has up to two parameters, use four lists to store the metrics
    reactive_active1 = []
    reactive_inactive1 = []
    treatment_active1 = []
    treatment_inactive1 = []
    
    reactive_active2 = []
    reactive_inactive2 = []
    treatment_active2 = []
    treatment_inactive2 = []
       
    type_dir = parent_dir + dirname+'/'
    print(type_dir)
    
    #第二層 列出前處理方式
    for dirname1 in os.listdir(type_dir):
        if (dirname1.endswith(".DS_Store")):
            continue
        
        pre_dir = type_dir + dirname1+'/'
        print(pre_dir)
        
        #第三層 列出reactive/treatment
        for dirname2 in os.listdir(pre_dir):
            if (dirname2.endswith(".DS_Store")):
                continue
            
            cond_dir = pre_dir + dirname2+'/'
            print(cond_dir)
            
            #第四層 列出active/inactive
            for dirname3 in os.listdir(cond_dir):
                if (dirname3.endswith(".DS_Store")):
                    continue
                    
                amd_dir = cond_dir + dirname3+'/'
                print(amd_dir)

                #列出目錄中的所有圖像
                for filename in os.listdir(amd_dir):
                    if not (filename.endswith(".bmp") or filename.endswith(".jpg")):
                        continue
                        
                    # 組合文件路徑
                    filepath = os.path.join(amd_dir, filename)
                    image = cv2.imread(filepath)
                    #print(filepath)
                    
                    #column名稱
                    column_name = dirname1+ '_'+ dirname2 + '_' + dirname3
                        
                    #initialize the parameters
                    img_parameter_1 = 0
                    img_parameter_2 = 0
                    image_name = filename.split('_')
                    
                    #分類：binary，parameter為entropy和VD
                    if(dirname == 'binary'):
                        # Read the image and calculate the entropy
                        img_parameter_1 = util2.calculate_image_entropy(filepath)
                        # Calculate the blood vessel density
                        img_parameter_2 = util2.calculate_bvd(image)
                        if(len(parameter)<3):
                            parameter.append("entropy")
                            parameter.append('vd')
                    
                    if(dirname == 'binary2'):
                        # calculate the blood vessel tortuosity
                        img_parameter_1 = util2.calculate_bvt(image)
                        # calculate the blood vessel caliber
                        img_parameter_2 = util2.calculate_bvc(image)
                        if(len(parameter)<3):
                            parameter.append("tortuosity")
                            parameter.append('caliber')
                    
                    if(dirname == 'faz'):
                        # calculate the blood vessel tortuosity
                        img_parameter_1 = util2.calculate_faz_ci(image)[0]
                        # calculate the blood vessel caliber
                        img_parameter_2 = util2.calculate_faz_ci(image)[1]
                        if(len(parameter)<3):
                            parameter.append("fazarea")
                            parameter.append('fazcirc') 
                    
                    if(dirname == 'binary3'):
                        # calculate the blood vessel tortuosity
                        img_parameter_1 = util2.calculate_vpi(image)
                        # calculate the blood vessel fractal dimension
                        img_parameter_2 = util2.calculate_bvc(image)
                        if(len(parameter)<3):
                            parameter.append("vpi")
                            parameter.append('bvc')
                    
                    if(dirname == 'canny' or 'gabor' or 'raw'):
                        # Read the image and calculate the entropy
                        img_parameter_1 = util2.calculate_image_entropy(filepath)
                        img_parameter_2 = util2.calculate_image_entropy(filepath)
                        if(len(parameter)<3):                         
                            parameter.append("entropy")
                            parameter.append('entropy')   
                            
                    # Add the results to the dictionary
                    #entropy[(column_name, image_name[0], image.shape[0], image.shape[1])] = img_entropy
                    if dirname2 == 'reactive':
                        if dirname3 == 'active':
                            reactive_active1.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_1))
                            reactive_active2.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_2))
                        if dirname3 == 'inactive':
                            reactive_inactive1.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_1)) 
                            reactive_inactive2.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_2))
                            
                    if dirname2 == 'treatment':
                        if dirname3 == 'active':
                            treatment_active1.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_1))
                            treatment_active2.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_2))
                        if dirname3 == 'inactive':
                            treatment_inactive1.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_1))
                            treatment_inactive2.append(((image_name[0], image.shape[0], image.shape[1], column_name), img_parameter_2))
                            
    # After reading all data, sort them by patient_number
    #先用方法分再用數字分
    n=0
    for a, b, c, d in ((reactive_active1, reactive_inactive1, treatment_active1, treatment_inactive1),(reactive_active2, reactive_inactive2, treatment_active2, treatment_inactive2)):
        for k in a,b,c,d:
            k.sort(key = lambda x: x[0][3])
            k.sort(key = lambda x: int(x[0][0]))

        dict_name_reactive = {}
        dict_name_treatment = {}
        
        #pair the active and inactive keys
        for (active_key, active_value), (inactive_key, inactive_value) in zip(a, b):
            pass
            dict_name_reactive[(inactive_key[0], inactive_key[3], active_key[0], active_key[3])] = [inactive_value, active_value]
        
        for (active_key, active_value), (inactive_key, inactive_value) in zip(c, d):
            pass
            dict_name_treatment[(active_key[0], active_key[3], inactive_key[0], inactive_key[3])] = [active_value, inactive_value]
            
        
        #print(dict_name_reactive)
            
        # Convert the results dictionary to a DataFrame with a multi-level index
        df_reactive = pd.DataFrame.from_dict(dict_name_reactive, orient='index')
        df_reactive.index = pd.MultiIndex.from_tuples(df_reactive.index, names=['inactive_number', 'inactive_process', 'active_number', 'active_process'])
        df_treatment = pd.DataFrame.from_dict(dict_name_reactive, orient='index')
        df_treatment.index = pd.MultiIndex.from_tuples(df_reactive.index, names=['active_number', 'active_process', 'inactive_number', 'inactive_process'])
        
        # Pivot the DataFrame to have directory names as columns
        df_reactive_inactive = df_reactive.pivot_table(values=0, index=['active_number','inactive_number'], columns=['inactive_process'])
        
        df_reactive_active = df_reactive.pivot_table(values=1, index=['active_number','inactive_number'], columns=['active_process'])
        df_treatment_active = df_treatment.pivot_table(values=0, index=['active_number','inactive_number'], columns=['active_process'])
        df_treatment_inactive = df_treatment.pivot_table(values=1, index=['active_number','inactive_number'], columns=['inactive_process'])
        
        # combine the two data frames using concat
        combined_df1 = pd.concat([df_reactive_inactive, df_reactive_active], axis = 1, ignore_index=False)
        combined_df2 = pd.concat([df_treatment_inactive, df_treatment_active], axis = 1, ignore_index=False)
        
        # Save the results to an Excel file
        excel_name1 = dirname + '_' + parameter[n] + '_reactive'
        excel_name2 = dirname + '_' + parameter[n] + '_treatment'
        combined_df1.to_excel('/home/charlie/projects/entropy/excel/'+ excel_name1+ '.xlsx')
        combined_df2.to_excel('/home/charlie/projects/entropy/excel/'+ excel_name2+ '.xlsx')
        
        #計次   
        n+=1  
    






/home/charlie/projects/entropy/input/raw/
/home/charlie/projects/entropy/input/raw/raw_superficial/
/home/charlie/projects/entropy/input/raw/raw_superficial/treatment/
/home/charlie/projects/entropy/input/raw/raw_superficial/treatment/active/
/home/charlie/projects/entropy/input/raw/raw_superficial/treatment/inactive/
/home/charlie/projects/entropy/input/raw/raw_superficial/reactive/
/home/charlie/projects/entropy/input/raw/raw_superficial/reactive/active/
/home/charlie/projects/entropy/input/raw/raw_superficial/reactive/inactive/
/home/charlie/projects/entropy/input/skeletonize/
/home/charlie/projects/entropy/input/skeletonize/gaussmean_skeletonize/
/home/charlie/projects/entropy/input/skeletonize/gaussmean_skeletonize/treatment/
/home/charlie/projects/entropy/input/skeletonize/gaussmean_skeletonize/treatment/active/
/home/charlie/projects/entropy/input/skeletonize/gaussmean_skeletonize/treatment/inactive/
/home/charlie/projects/entropy/input/skeletonize/gaussmean_skeletonize/reactive

error: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/contours.cpp:195: error: (-210:Unsupported format or combination of formats) [Start]FindContours supports only CV_8UC1 images when mode != CV_RETR_FLOODFILL otherwise supports CV_32SC1 images only in function 'cvStartFindContours_Impl'
